# Federated Learning Project

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

W0810 16:09:04.909571 140259689711488 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0810 16:09:04.923770 140259689711488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.log_interval = 10
        self.save_model = False

args = Arguments()






In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

In [0]:
# TODO: Define your network architecture here
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [0]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
#         data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
if __name__ == '__main__':
  import multiprocessing as mp
  mp.set_start_method('forkserver', force=True)
#   model = Net().to(device)
  model = Net()
  optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

  for epoch in range(1, args.epochs + 1):
      train(args, model, federated_train_loader, optimizer, epoch)
      test(args, model, test_loader)

  if (args.save_model):
      torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.322450
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.321743
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.307015
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.297934
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.309897
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.298327
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.308297
Train Epoch: 1 [4480/60032 (7%)]	Loss: 2.267097
Train Epoch: 1 [5120/60032 (9%)]	Loss: 2.284019
Train Epoch: 1 [5760/60032 (10%)]	Loss: 2.282748
Train Epoch: 1 [6400/60032 (11%)]	Loss: 2.295989
Train Epoch: 1 [7040/60032 (12%)]	Loss: 2.262578
Train Epoch: 1 [7680/60032 (13%)]	Loss: 2.250473
Train Epoch: 1 [8320/60032 (14%)]	Loss: 2.250357
Train Epoch: 1 [8960/60032 (15%)]	Loss: 2.240908
Train Epoch: 1 [9600/60032 (16%)]	Loss: 2.228352
Train Epoch: 1 [10240/60032 (17%)]	Loss: 2.222480
Train Epoch: 1 [10880/60032 (18%)]	Loss: 2.196472
Train Epoch: 1 [11520/60032 (19%)]	Loss: 2.201719
Train Epoch: 1 [12160/60032 (20%)]	Loss: 2.180597
Train Epoch: 1 [12800/60032 (